In [1]:
# Core
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline

# Astronomy
import astropy
from astropy.units import au, minute

In [2]:
# Local imports
import kepler_sieve
from asteroid_data import load_ast_elements, load_ast_vectors
from asteroid_spline import get_df_shape, spline_ast_data, make_spline_df
from asteroid_direction import calc_dir_ast2obs
from astro_utils import deg2dist, dist2sec
from orbital_element import elt2pos
from planets_interp import get_sun_pos
from db_utils import sp2df, df2db

In [27]:
from detection_near_ast import get_data_detections, get_data_ast_dir, asteroid_batch_prelim, asteroid_batch_vec, asteroid_batch_elt, light_time_adj, make_ast_num_df
from detection_near_ast import cols_u_obs, cols_q_obs, cols_u_ast, cols_v_ast, cols_q_ast, mjd0, mjd1, c

In [28]:
# Inputs
d0 = 1
d1 = 1000
n0 = 1
n1 = 10
deg_max = 180
arcmin_max = deg_max * 60

In [29]:
# Inputs
d0 = 37501
d1 = 38501
n0 = 1
n1 = 3
deg_max = 180
arcmin_max = deg_max * 60

In [30]:
# Get detections
det = get_data_detections(d0=d0, d1=d1)
det

,DetectionID,tObs,uObs_x,uObs_y,uObs_z,qObs_x,qObs_y,qObs_z
DetectionID,,,,,,,,
37501,37501,58272.165440,-0.937908,0.346871,0.003123,-0.310321,-0.958524,-0.00003
37502,37502,58272.165440,-0.937899,0.346895,0.003122,-0.310321,-0.958524,-0.00003
37503,37503,58272.165440,-0.937874,0.346961,0.003327,-0.310321,-0.958524,-0.00003
37504,37504,58272.165440,-0.937833,0.347070,0.003455,-0.310321,-0.958524,-0.00003
37505,37505,58272.165440,-0.937852,0.347015,0.003787,-0.310321,-0.958524,-0.00003
...,...,...,...,...,...,...,...,...
38496,38496,58272.166377,-0.887159,0.266294,0.376876,-0.310306,-0.958529,-0.00003
38497,38497,58272.166377,-0.887357,0.265446,0.377008,-0.310306,-0.958529,-0.00003
38498,38498,58272.166377,-0.886836,0.270057,0.374955,-0.310306,-0.958529,-0.00003


In [14]:
# First pass at detections near asteroids
dna_pre = asteroid_batch_prelim(det=det, n0=n0, n1=n1, arcmin_max=arcmin_max)

In [15]:
dna_pre

,DetectionID,AsteroidID,tObs,uObs_x,uObs_y,uObs_z,qObs_x,qObs_y,qObs_z,tAst,uAst_x,uAst_y,uAst_z,s,LightTime
0,1,1,58270.167662,-0.940203,-0.205183,0.271880,-0.342267,-0.947311,-0.000032,58270.151886,-0.750592,0.638408,0.170429,0.870569,22.716823
1,2,2,58270.167662,-0.940593,-0.207164,0.269011,-0.342267,-0.947311,-0.000032,58270.150803,-0.096493,0.916145,-0.389060,1.551576,24.277300
2,3,1,58270.167662,-0.942678,-0.201138,0.266274,-0.342267,-0.947311,-0.000032,58270.151886,-0.750592,0.638408,0.170429,0.866557,22.716823
3,4,2,58270.167662,-0.944177,-0.198029,0.263275,-0.342267,-0.947311,-0.000032,58270.150803,-0.096493,0.916145,-0.389060,1.544504,24.277300
4,1,1,58270.167662,-0.940203,-0.205183,0.271880,-0.342267,-0.947311,-0.000032,58270.151886,-0.750592,0.638408,0.170429,0.870569,22.716823
5,2,2,58270.167662,-0.940593,-0.207164,0.269011,-0.342267,-0.947311,-0.000032,58270.150803,-0.096493,0.916145,-0.389060,1.551576,24.277300
6,3,1,58270.167662,-0.942678,-0.201138,0.266274,-0.342267,-0.947311,-0.000032,58270.151886,-0.750592,0.638408,0.170429,0.866557,22.716823
7,4,2,58270.167662,-0.944177,-0.198029,0.263275,-0.342267,-0.947311,-0.000032,58270.150803,-0.096493,0.916145,-0.389060,1.544504,24.277300


In [16]:
# Sharpen estimate using splined asteroid vectors
dna_vec = dna_pre.copy()
asteroid_batch_vec(dna=dna_vec, n0=n0, n1=n1, arcmin_max=arcmin_max)

In [17]:
dna_vec

,DetectionID,AsteroidID,tObs,uObs_x,uObs_y,uObs_z,qObs_x,qObs_y,qObs_z,tAst,uAst_x,uAst_y,uAst_z,s,LightTime,qAst_x,qAst_y,qAst_z
0,1,1,58270.167662,-0.940203,-0.205183,0.271880,-0.342267,-0.947311,-0.000032,58270.151902,-0.750296,0.638716,0.170581,0.870914,22.694705,-2.389672,0.795615,0.465448
1,2,2,58270.167662,-0.940593,-0.207164,0.269011,-0.342267,-0.947311,-0.000032,58270.150809,-0.095533,0.916172,-0.389234,1.552191,24.267722,-0.621027,1.726018,-1.135791
2,3,1,58270.167662,-0.942678,-0.201138,0.266274,-0.342267,-0.947311,-0.000032,58270.151902,-0.750296,0.638716,0.170581,0.866904,22.694705,-2.389672,0.795615,0.465448
3,4,2,58270.167662,-0.944177,-0.198029,0.263275,-0.342267,-0.947311,-0.000032,58270.150809,-0.095533,0.916172,-0.389234,1.545124,24.267722,-0.621027,1.726018,-1.135791
4,1,1,58270.167662,-0.940203,-0.205183,0.271880,-0.342267,-0.947311,-0.000032,58270.151902,-0.750296,0.638716,0.170581,0.870914,22.694705,-2.389672,0.795615,0.465448
5,2,2,58270.167662,-0.940593,-0.207164,0.269011,-0.342267,-0.947311,-0.000032,58270.150809,-0.095533,0.916172,-0.389234,1.552191,24.267722,-0.621027,1.726018,-1.135791
6,3,1,58270.167662,-0.942678,-0.201138,0.266274,-0.342267,-0.947311,-0.000032,58270.151902,-0.750296,0.638716,0.170581,0.866904,22.694705,-2.389672,0.795615,0.465448
7,4,2,58270.167662,-0.944177,-0.198029,0.263275,-0.342267,-0.947311,-0.000032,58270.150809,-0.095533,0.916172,-0.389234,1.545124,24.267722,-0.621027,1.726018,-1.135791


In [18]:
(dna_vec.tAst - dna_pre.tAst)*1440

0    0.022118
1    0.009578
2    0.022118
3    0.009578
4    0.022118
5    0.009578
6    0.022118
7    0.009578
Name: tAst, dtype: float64

In [21]:
# (dna_vec.LightTime - dna_pre.LightTime)

In [23]:
(dna_vec.uAst_x - dna_pre.uAst_x)*3600*60

0     63.967716
1    207.229501
2     63.967716
3    207.229501
4     63.967716
5    207.229501
6     63.967716
7    207.229501
Name: uAst_x, dtype: float64

In [24]:
(dna_vec.uAst_y - dna_pre.uAst_y)*3600*60

0    66.449872
1     5.777071
2    66.449872
3     5.777071
4    66.449872
5     5.777071
6    66.449872
7     5.777071
Name: uAst_y, dtype: float64